In [1]:
import os
import time
import json
import logging
from tqdm import tqdm
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import google.generativeai as genai
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def initialize_edge_driver():
    edge_options = Options()
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--headless")
    service = Service(r'C:/Users/PedroLustosa/OneDrive - Grupo Portfolio/Documentos/edgedriver_win64/msedgedriver.exe')
    driver = webdriver.Edge(service=service, options=edge_options)
    return driver

def get_soup_xpath(url, xpath1, xpath2=None):
    driver = None
    soup = None
    
    # Tente encontrar o primeiro XPath
    try:
        driver = initialize_edge_driver()
        driver.get(url)
        element1 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath1))
        )
        if element1:
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            return soup
    except:
        pass
    finally:
        if driver:
            driver.quit()
            
    # Tente encontrar o segundo XPath, se fornecido
    if xpath2:
        try:
            driver = initialize_edge_driver()
            driver.get(url)
            element2 = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath2))
            )
            if element2:
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                links = soup.find_all('a', class_='bg-gray-l2 ds-8 border-0 rounded-0 mb-3 nav-btn nav-btn-horizontal mb-3 noLinking noDecoration card')
                filtered_links = [f"https://www.cdc.gov{link['href']}" for link in links if 'illness' in link['href'] or 'symptoms' in link['href']]
                href_list_card.extend(filtered_links)
        except:
            pass
        finally:
            if driver:
                driver.quit()         
    return soup

def extract_text_by_class(soup, class_name):
    if soup:
        container = soup.find(class_=class_name)
        if container:
            elements = container.find_all(['h1', 'h2', 'h3', 'p', 'li', 'ul'])
            text = '\n'.join(element.get_text(strip=True) for element in elements)
        else:
            text = ""
    else:
        text = ""
    return text

def generate_text(prompt, model):
    json_doencas = {}
    if prompt:
        try:
            response = model.generate_content(prompt)
            response_text = response.text.replace("```", "").replace("json\n", "").strip()
            json_doencas = json.loads(response_text)
        except json.JSONDecodeError as e:
            pass
        except Exception as e:
            pass
    return json_doencas

def configure_genai_model():
    load_dotenv()
    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
    genai.configure(api_key=GEMINI_API_KEY)
    generation_config = {
        "temperature": 0.4,
    }
    safety_settings = {
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL': 'BLOCK_NONE',
        'DANGEROUS': 'BLOCK_NONE'
    }
    instruction = """Você está atuando como um cientista de dados sênior. Sua tarefa é extrair informações sobre doenças e sintomas em português Brasil a partir do texto fornecido. Identifique e liste cada doença junto com seus sintomas associados conforme descrito no texto. Formate o resultado como um objeto JSON.

    Especificações:
    Identificação de Doenças: Enumere cada doença mencionada no texto.
    Identificação de Sintomas: Para cada doença, liste os sintomas associados conforme descrito no texto.
    Formato JSON: Organize as informações no formato JSON conforme o exemplo abaixo.

    Exemplo de Formatação JSON:
    {
        "HIV": [
            "Febre",
            "Calafrios",
            "Rash",
            "Suores noturnos (sudoração intensa durante o sono)",
            "Dores musculares",
            "Dor de garganta",
            "Fadiga",
            "Inchaço dos gânglios linfáticos",
            "Úlceras na boca"
        ],
        "Mielite Flácida Aguda": [
            "Fraqueza nos braços ou pernas",
            "Perda de tônus muscular e reflexos",
            "Queda ou fraqueza facial",
            "Dificuldade para mover os olhos",
            "Pálpebras caídas",
            "Dificuldade para engolir",
            "Dificuldade para falar",
            "Dor nos braços, pernas, costas ou pescoço"
        ]
    }
    """
    model = genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        generation_config=generation_config,
        safety_settings=safety_settings,
        system_instruction=instruction
    )
    return model

def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def save_to_csv(file_name, data):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['URL'])
        for item in data:
            writer.writerow([item])      

In [3]:
# Configura o modelo
model = configure_genai_model()

# URL base
base_url = 'https://www.cdc.gov/health-topics.html#A'
xpath = '//*[@id="cdc-atozlist"]/div/div[3]/div/ul/li[1]/a'

# Obtém os links da página inicial
soup = get_soup_xpath(base_url, xpath)
link_hrefs = soup.select('#cdc-atozlist div div ul li a')
save_to_csv('links.csv', link_hrefs)

# Dicionário para armazenar informações de doenças
doencas = {}
href_list_card = []

Fetching:  76%|███████▋  | 39/51 [34:45<33:32, 167.72s/Links]  

In [ ]:
# Itera pelos links e extrai informações
for ref in tqdm(link_hrefs[:51], desc="Fetching", unit="Links"):
    soup_link = get_soup_xpath(ref.get("href"), '//*[@id="content"]/div[1]','/html/body/div[4]/main/div[3]/div/div[4]/div/div[1]/div[2]')
    texto = extract_text_by_class(soup_link, 'cdc-dfe-body__center')
    json_doenca = generate_text(texto, model)
    doencas.update(json_doenca)
    time.sleep(5)

# Salva 
save_to_json(doencas, 'doencas_cdc_50.json')
save_to_csv('links_card_50.csv', href_list_card)

In [ ]:
save_to_csv('links_card_50.csv', href_list_card)
save_to_csv('links.csv', link_hrefs)